In [2]:
from haystack.document_stores import InMemoryDocumentStore
import time
start_time=time.time()
document_store = InMemoryDocumentStore(use_bm25=True)

doc_dir = "CleanData"


import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

Updating BM25 representation...: 100%|██████████| 162/162 [00:00<00:00, 50620.37 docs/s]


In [7]:
prediction = pipe.run(
    query="What do I do if the customer does not respond",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 1}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
) 

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 13.24 Batches/s]

'Query: What do I do if the customer does not respond'
'Answers:'
[   {   'answer': 'we apologize for not being able to further assist and '
                  "provide the dealer number I'm sorry, I am unable to assist",
        'context': 'ntinue in English, we apologize for not being able to '
                   "further assist and provide the dealer number I'm sorry, I "
                   'am unable to assist. Feel free to repl'}]
